# Chap7:data type, class
型や型クラスを自分で作る

値コンストラクタを複数組み合わせてデータ型を作る

In [8]:
data Shape= Circle Float Float Float |
                       Rectangle Float Float Float Float
         deriving(Show)

値コンストラクタはただの関数である。

In [6]:
:t Rectangle

Rectangle :: Float -> Float -> Float -> Float -> Shape

In [5]:
area :: Shape -> Float
area (Circle _ _ r) = pi * r ^2
area (Rectangle x1 y1 x2 y2) = abs ( x2-x1) * abs ( y2-y1)

In [7]:
area $ Circle 10 20 10

314.15927

## レコード構文


In [10]:
data Person = Person { firstName :: String
                                            , lastName :: String
                                            , age :: Int
                                            , height :: Float
                                            , phoneNumber :: String
                                            , flavor :: String } deriving (Show)

In [11]:
:t firstName

firstName :: Person -> String

In [12]:
Person {firstName="Anna",lastName="Stanford", age=18,height=168,phoneNumber="110-202",flavor="berry"}

Person {firstName = "Anna", lastName = "Stanford", age = 18, height = 168.0, phoneNumber = "110-202", flavor = "berry"}

## 型引数
値コンストラクタ（Circleとか）は引数をとって新しい型の値を生み出す。型コンストラクタは型引数を引数にとって新しい型を生み出す。MaybeのJustとか。(Justという型は存在できずJust a(Int,Charなど)という型が存在する。Justは型コンストラクタでaは型引数。)リスト`[]`とか。
- 具体型：型引数を一つも取らない値や型引数が全て埋まっている型。値はすべて具体型。

In [16]:
:t Just "boo"
:t Nothing
:t ["foo","bar","boo"]
:t []

Just "boo" :: Maybe [Char]

Nothing :: forall a. Maybe a

["foo","bar","boo"] :: [[Char]]

[] :: forall a. [a]

Nothingの型はMaybe a(多相的)、空リストの型も多相的。
## インスタンスの自動導出
### 復習:型クラス
型クラスのインスタンスである型は、その型クラスが記述する振る舞いを実装する。
具体的には型クラスはメソッドという関数をもち、ある型が型クラスのインスタンスである時、それらのメソッドがその型ではどういう意味を成すのかを定義する。
### インスタンスの自動導出 (derive)
Eq,Ord,Enum,Bounded,Show,Readの型クラスの文脈での振る舞いを自動導出
（つまりこれらの型クラスが持っているメソッドが自動で使えるようになる）

## 型シノニム
`[Char]`と`String`は同値で交換可能

In [17]:
type PhoneBook = [(String,String)]

### 型シノニムの多相化
型シノニムも型引数を取るようにできる。

In [19]:
type AssocList k v =[(k,v)]

### 再帰的なデータ構造
`:-:`は中置関数(`:`で始まる必要がある)

In [25]:
infixr 5 :-:
data List a = Empty |  a :-:  (List a) deriving(Show,Read,Eq,Ord)
3 :-: 4 :-: 5 :-: Empty
infixr ^++
(^++)  :: List a -> List a-> List a
Empty ^++ ys = ys
(x:-:xs) ^++ ys = x :-: (xs ^++ ys ) --パターンマッチを値コンストラクタに対して使う
(3 :-: 4 :-: 5 :-: Empty) ^++ (1 :-: 2 :-: 3 :-: Empty)

3 :-: (4 :-: (5 :-: Empty))

3 :-: (4 :-: (5 :-: (1 :-: (2 :-: (3 :-: Empty)))))

## 独自の型クラスを作り、そのインスタンスを手動で作る
### 既存の型クラスのインスタンスを手動で作る

In [1]:
data TrafficLight = Red | Yellow | Green
instance Eq TrafficLight where
    Red == Red = True
    Green == Green = True
    Yellow == Yellow = True
    _ == _ = False
instance Show TrafficLight where
    show Red = "Red light"
    show Green = "Green light"
    show Yellow = "Yellow light"
    

In [3]:
Red == Red
Red == Yellow
Red `elem` [Red,Yellow,Green]
[Red,Yellow,Green]

True

False

True

[Red light,Yellow light,Green light]

### 別の型クラスのサブクラスである型クラスを作る
型クラス宣言に型クラス制約をかける
### 多相型を型クラスのインスタンスに
 多相型のまま型クラスのインスタンスにできる。
 ```
instance (Eq m) => Eq (Maybe m) where
    Just x == Just y = x == y
    Nothing == Nothing = True
    _ == _ = False 
```
mがEqに属している場合のみ、Maybe mの形をしている型を全てEqに属するようにする
### 独自のYesNo型クラスとそれに属するインスタンスである型

In [5]:
class YesNo a where
    yesno :: a->Bool
instance YesNo Int where
    yesno 0 = False
    yesno _ = True
instance YesNo [a] where
    yesno [] = False
    yesno _ = True
instance YesNo Bool where
    yesno = id --引数を一つとって同じものを返すだけの標準ライブラリ
instance YesNo (Maybe a) where
    yesno (Just _) = True
    yesno Nothing = False
instance YesNo TrafficLight where
    yesno Red=False
    yesno _ = True

In [10]:
yesno $ length []
yesno "haha"
yesno $ Just 0
yesno True
yesno Red
yesno [0,0,0]
:t yesno

False

True

True

True

False

True

yesno :: forall a. YesNo a => a -> Bool

## Functor型クラス
 Functor型クラスに属しているインスタンスはリスト・Maybe型など
```
class Functor f where
    fmap :: (a->b) -> f a -> f b
```
※fは一つの型をとる型コンストラクタ
```
instance Functor Maybe where
    fmap f (Just x) = Just (f x)
    fmap f Nothing = Nothing
```
## 型の種類(kind)
```
:k (型の種類)
```
で調べられる。

In [16]:
:k Int
:k Maybe
:k Maybe Int
:k Either

Int :: *

Maybe :: * -> *

Maybe Int :: *

Either :: * -> * -> *